<center><h1>Reading scholarly.html files</h1></center>

## Introduction

In this notebook I am going to attempt to write a parser to read scholarly.html files.

### Find an example file and load it

In [1]:
import os

print(os.listdir('./results/PMC2241601'))

html_root = './results/PMC2241601/scholarly.html'

#load the data in
with open(html_root, 'r') as f:

    raw_text = f.read()
    


['svg', 'word.frequencies.snippets.xml', 'fulltext.pdf', 'pdfimages', 'search.country.snippets.xml', 'scholarly.html', 'eupmc_result.json', 'search.country.count.xml', 'results', 'fulltext.xml', 'word.frequencies.count.xml']


In [2]:
from bs4 import BeautifulSoup

#parse the html into soup
soup = BeautifulSoup(raw_text, 'html.parser')

#extract the body
body = soup.find('body')



### Try and make the parser work using just the abstract

In [3]:
#extract the front element
front = body.find('div', {'class':'front'})

In [4]:
### Extract the journal metadata

#extract the journal metadata
journal_meta = front.find('div', {'class':'journal-meta'})

#extract the journal title
journal_title = journal_meta.find('span', {'class':'journal-title'}).text

#extract the journal issn value
journal_issn = journal_meta.find('span', {'class':'issn-epub'}).text

#extract the journal publisher element
journal_publisher_el = journal_meta.find('div', {'class':'publisher'})

#extract the journal publisher name
journal_publisher_name = journal_publisher_el.find('span', {'class':'publisher-name'}).text

#extract the paper metadata
paper_meta = front.findAll('div', {'class':'article-meta'})

#lets have a look at what we've got here
journal_meta.prettify()


'<div class="journal-meta" tagx="journal-meta" title="journal-meta">\n <span class="nlm-ta" title="nlm-ta">\n  BMC Plant Biol\n </span>\n <span class="journal-title" tagx="journal-title" title="journal-title">\n  BMC Plant Biology\n </span>\n <span class="issn-epub" tagx="issn" title="issn-epub">\n  1471-2229\n </span>\n <div class="publisher" tagx="publisher" title="publisher">\n  <span class="publisher-name" tagx="publisher-name" title="publisher-name">\n   BioMed Central\n  </span>\n </div>\n</div>\n'

In [5]:
#journal_meta.find('span', {'class':'journal-title'})

journal_publisher_el


<div class="publisher" tagx="publisher" title="publisher"><span class="publisher-name" tagx="publisher-name" title="publisher-name">BioMed Central</span></div>

### Extract the paper metadata

In [6]:
art_meta_el = front.find('div', {'class':'article-meta'})

#get the publisher id
pub_id = art_meta_el.find('span', {'class':'publisher-id'}).text

#get the pmid
pmid = art_meta_el.find('span', {'class':'pmid'}).text


In [7]:
class recursion_tests(object):
    
    def __init__(self, front_el):
        
        self.store_json = {}
        
        self.path = []
        
        el_label = front_el['class'][0]
        
        self.recursive_search(front_el, el_label)
        
        '''
        for ci, child in enumerate(front_el.children):
            
            #check if its one of those wierd NavigableString things
            if 'bs4.element.NavigableString' in str(child.__class__):
                #move on if it does
                continue
        
            if child.name == 'div':
                
                store = self.recursive_search(child)
        '''
        
    def recursive_search(self, div_el, top_layer):
        
        div_el_title = div_el['class'][0]
        
        #store_obj = {div_el_title:{}}
        top_layer[div_el_title] = {}
        
        for chi, child1 in enumerate(store_obj.children):
            
            #check if its one of those wierd NavigableString things
            if 'bs4.element.NavigableString' in str(child.__class__):
                #move on if it does
                continue
                
            if child1.name == 'div':
                
                self.path.append(child1['class'][0])
                print(path)
                
                store = self.recursive_search(child1)
                
            elif child1.name == 'span':
                
                #extract the data from the span element we've found
                title = child1['class'][0]
                text = child1.text
                
                store_obj[div_el_title][title] = text
                
                

##################################
### End of function defintions ###
##################################


'''
### Reference implementation
json_dict = {}
flat_dict = {}

for parent, children in file_iterator():
    if parent in flat_dict:
        value = flat_dict[parent]
    else:
        value = {}
        flat_dict[parent] = json_dict[parent] = value
    for child in children:
        flat_dict[child] = value[child] = {}
'''

tester = recursion_tests(front.find('div', {'class':'journal-meta'}))

SyntaxError: invalid syntax (<ipython-input-7-1568a7d40180>, line 30)

In [13]:
from pprint import pprint

def iterdict(d):
    for k,v in d.items():        
        if isinstance(v, dict):
            iterdict(v)
        else:            
            print (k,":",v)
        
def iterhtml(el):
    
    store_json = {}
    
    if el.children:
    
        for item in el.children:
            
            #check if the item is whatever a navigable string is
            if 'bs4.element.NavigableString' in str(item.__class__):
                #move on if it does
                continue

            if item.name == 'div':
                print('going down')
                
                store_json[item['class'][0]] = iterhtml(item)
                
            elif item.name == 'span':
                
                store_json[item['class'][0]] = item.text
                
    return store_json


result = iterhtml(front)
    
print('++++++++')
    
pprint(result)

going down
going down
going down
going down
going down
going down
going down
going down
going down
going down
going down
going down
going down
going down
going down
going down
++++++++
{'article-meta': {'abstract': {'abstract-title': {},
                               'background': {'title': {}},
                               'conclusion': {'title': {}},
                               'results': {'title': {}}},
                  'article-categories': {'subject': 'Research Article'},
                  'citation_author_institution': '[2], ',
                  'contrib-group': {'contrib': 'WesthoffPeter1west@uni-duesseldorf.de'},
                  'doi': ' doi: 10.1186/1471-2229-8-4',
                  'fpage': '4',
                  'history': 'received: 2007-11-8accepted: 2008-1-21',
                  'lpage': '4',
                  'permissions': {'copyright': '(C) Engelmann et al; licensee '
                                               'BioMed Central Ltd., 2008',
                 

In [11]:
for child in front.children:
    
    print(child)

 
<div class="journal-meta" tagx="journal-meta" title="journal-meta"><span class="nlm-ta" title="nlm-ta">BMC Plant Biol</span><span class="journal-title" tagx="journal-title" title="journal-title">BMC Plant Biology</span><span class="issn-epub" tagx="issn" title="issn-epub">1471-2229</span><div class="publisher" tagx="publisher" title="publisher"><span class="publisher-name" tagx="publisher-name" title="publisher-name">BioMed Central</span></div> </div>
 
<div class="article-meta" tagx="article-meta" title="article-meta"><span class="publisher-id" title="publisher-id">1471-2229-8-4</span><span class="pmid" title="pmid"> pmid: <a href="http://www.ncbi.nlm.nih.gov/pubmed/18208593">18208593</a></span><span class="doi" title="doi"> doi: <a href="https://dx.doi.org/10.1186/1471-2229-8-4">10.1186/1471-2229-8-4</a></span><div class="article-categories" title="article-categories">
: <span class="subject" title="subject">Research Article</span></div> <div class="title-group" tagx="title-group" 

In [ ]:

store_json = {}
for child in front.children:
    
    
    if 'bs4.element.NavigableString' in str(child.__class__):
        
        continue
        
    print(child.name)
    
    for subchild in child.children:
        
        print(subchild)
        if subchild.name == 'span':
            
            print('SPAN2')
            
        elif subchild.name == 'div':
            
            print('DIV2')
            print("===============================")
            for subsubchild in subchild.children:
                
                
                print(subsubchild)
                
                if subsubchild.name == 'span':
                    
                    print('SPAN3')
                elif subsubchild.name == 'div':
                    
                    print('DIV3')
        
            print('===============================')
        
        print('---')
        
    print()
    print('++++++++++++++++++')


In [ ]:
'''
def process_front(front_el):
    
    for child in front_el.children:

        print(child)
        #print()
        print(type(child))
        print()

        if 'bs4.element.NavigableString' in str(child.__class__):

            continue

        print(child.name)

        for subchild in child.children:

            print(subchild)
            if subchild.name == 'span':

                print('SPAN')
                
'''

In [ ]:
child.__class__

In [ ]:
from pprint import pprint

els = art_meta_el.children
meta_store = {}
for el in els:
    
    if 'NavigableString' in str(type(el)):
        continue
    
    if el.text.strip() == '':
        continue
    #print(el)
    
    if el['class'][0] == 'contrib-group':
        
        #extract the list of authors
        people = el.findAll('div', {'class':'contrib'})
        
        for person in people:
            
            #extract the person's email
            email = person.find('span', {'class':'email'})
            
            #extract the persons surname
            surname = person.find('')
    
    meta_store[el['class'][0]] = el.text.split(':')[-1].strip()
    
pprint(meta_store)

In [ ]:
str(type(el))

In [ ]:
pprint(meta_store)

### Extract the abstract

In [ ]:
#find the abstract
abstract = front.find('div', {'class':'abstract'})

#get the abstract title
abstract_title = abstract.find('div', {'class':'abstract-title'}).text.strip()

#remove the title section
abstract.find('div', {'class':'abstract-title'}).decompose()

#get the abstract text
abstract_text = abstract.text.strip()


#print('TITLE')
#print(abstract_title)
#print('ABSTRACT BODY')
#print(abstract_text)

### Extract the references etc.

In [ ]:
#Extract the references

#pull the back
back = body.find('div', {'class':'back'})

#get the Acknowledgements
ack = back.find('div', {'class':'ack'})

#get the references
refs = back.find('div', {'tag':'ref-list'})

print(refs)